In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import mne.io as mio

In [3]:
start = 103
end = 203

In [ ]:
sounds = []
for i in range (start, end):
    temp = []
    raw = mio.read_raw_edf("../dataset/2_Test/Test_" + str(i).zfill(3) + ".edf")
    for num in range(1, 21):
        temp.append(raw.get_data(picks="Mel_Freq" + str(num).zfill(2))[0])
    sounds.append(temp)

Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
339.14 MiB


/miniconda/envs/empty-maic2022/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
1598.68 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_003.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
2514.48 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_004.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
3438.45 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_005.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
3988.02 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_006.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
4789.14 MiB
Extracting EDF parameters from /home/maic-pla

Creating raw.info structure...
36078.59 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_045.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
36627.90 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_046.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
37612.12 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_047.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
38722.20 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_048.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
39637.75 MiB
Extracting EDF parameters from /home/maic-player/dataset/3_Private Test/Private_Test_049.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
40713.53 

In [ ]:
from sklearn import preprocessing

for i in range(0, len(sounds)):
    print("하고 있다고요!!!!")
    for j in range(0, 20):
        sounds[i][j] = (preprocessing.normalize([sounds[i][j]]))
        sounds[i][j] = sounds[i][j].squeeze()

In [ ]:
clustered_people_sound = []
clustered_person_sound = []
clustered_channel = []

for i in range(0, len(sounds)):
    print("I'm trying my best...")
    for j in range(0, 20):
        for k in range(0, len(sounds[i][j]), 500):
            clustered_channel.append(np.mean(sounds[i][j][k : k + 500]))
        clustered_person_sound.append(clustered_channel)
        clustered_channel = []
    clustered_people_sound.append(clustered_person_sound)
    clustered_person_sound = []

In [ ]:
people = []
for i in range (start, end):
    temp = []
    raw = mio.read_raw_edf("../dataset/2_Test/Test_" + str(i).zfill(3) + ".edf")
    temp.append(raw.get_data(picks="X axis")[0])
    temp.append(raw.get_data(picks="Y axis")[0])
    temp.append(raw.get_data(picks="Z axis")[0])
    people.append(temp)

In [ ]:
#clustering
clustered_people = []
clustered_person = []
for person in people:
    print("Doing... plz wait ㅠㅠ")
    clustered_person = []
    for axis in person:
        temp = []
        for location in range(0, len(axis), 500):
            temp.append(np.mean(axis[location : location + 500]))
        clustered_person.append(temp)
    clustered_people.append(clustered_person)

In [ ]:
location_people_xyz = []

# 각 사람에 대해
for person in clustered_people:
    print("Doing!! GOGO!")
    one_person_xyz = []
    #각 축에 대해
    for axis in person:
        length = len(axis)
        ten_len = (int)(length / 10)
        location_xyz = []
        #축의 10분의 1씩 나눠서 분석
        for j in range(0, length - ten_len, ten_len):
            for k in range (j, j + ten_len):
                std = np.std(axis[j : j + ten_len])
                if abs(axis[k] - axis[k + 1]) > 1 * std:
                    location_xyz.append(k)
        one_person_xyz.append(location_xyz)
    location_people_xyz.append(one_person_xyz)

In [ ]:
# 사람수 만큼 반복
limit = 10

all_person = []
for i in range (0, len(clustered_people_sound)):
    print(i)
    # 각 사람의 20개 채널에 대해
    all_channel = []
    for j in range(0, 20):
        cnt = 0
        length = len(clustered_people_sound[i][j])
        ten_len = (int)(length / 10)
        # 채널을 잠 잔 시간의 10분의 1씩 나눠서 분석
        for k in range(0, length - ten_len, ten_len):
            std = np.std(clustered_people_sound[i][j][k : k + ten_len])
            for z in range(k, k + ten_len):
                if (abs(clustered_people_sound[i][j][z] - clustered_people_sound[i][j][z+1]) > 1 * std):
                    add = 1
                    for l in range (0, len(location_people_xyz[i][0])):
                        if (location_people_xyz[i][0][l] - limit < z < location_people_xyz[i][0][l] + limit):
                            add = 0
                    for l in range (0, len(location_people_xyz[i][1])):
                        if (location_people_xyz[i][1][l] - limit < z < location_people_xyz[i][1][l] + limit):
                            add = 0
                    for l in range (0, len(location_people_xyz[i][2])):
                        if (location_people_xyz[i][2][l] - limit < z < location_people_xyz[i][2][l] + limit):
                            add = 0
                    cnt += add
        all_channel.append(cnt / length * 100)
    all_person.append(all_channel)

In [ ]:
all_person

In [ ]:
import pickle

file_name = "Data/Private_Test_sound"
with open(file_name, "wb") as f:
    pickle.dump(all_person, f)